In [5]:
import sys
sys.path.append('/home/tcalo3/projects/project6')
sys.path.append('/home/tcalo3/projects/project6/src/modules/components/ganspace')

In [6]:
# Load model
from IPython.utils import io
import ipywidgets as widgets
from PIL import Image
import imageio
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte
from PIL import Image
import numpy as np
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import ExpectedImprovement
from botorch.acquisition.monte_carlo import qUpperConfidenceBound
import warnings
import numpy as np
import math
from botorch.fit import fit_gpytorch_model
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.models.pairwise_gp import PairwiseGP, PairwiseLaplaceMarginalLogLikelihood

warnings.filterwarnings('ignore')

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Specify model to use
config = Config(
  model='StyleGAN',
  layer='g_mapping.dense7',
  output_class='ffhq',
  components=32,
  use_w=True,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)

model = inst.model

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


In [67]:
from src.modules.metrics.models.facenet import FaceNetModel

def sample_pca_space(n_samples, pca_components, pca_stdevs):
    random_samples = np.random.randn(n_samples, pca_components) * pca_stdevs
    return random_samples

def reconstruct_from_pca(pca_samples, pca_components, data_mean):
    original_data = np.dot(pca_samples, pca_components) + data_mean
    return original_data


def project_to_pca(new_data, pca_components, data_mean):
    centered_data = new_data - data_mean
    pca_projection = np.dot(centered_data, pca_components.T)
    return pca_projection

with torch.set_grad_enabled(False):
    facenet = FaceNetModel().eval()
    pass


def reconstruct_and_evaluate(X,target,latent_dirs,latent_stdevs,eval_model):
    target_out = model.sample_np(target)
    target_image = Image.fromarray((target_out * 255).astype(np.uint8))
    target_resized_image = target_image.resize((160, 160), Image.NEAREST)
    target_tensor = torch.from_numpy(np.array(target_resized_image)).unsqueeze(0).permute(0, 3, 1, 2)
    target_embedding = facenet(target_tensor.float())

    X = reconstruct_from_pca(X,comps['lat_comp'][:,0,:].T,comps['lat_mean'])
    X_out = model.sample_np(X.astype(np.float32))
    X_image = Image.fromarray((X_out * 255).astype(np.uint8))
    X_resized_image = X_image.resize((160, 160), Image.NEAREST)
    X_tensor = torch.from_numpy(np.array(X_resized_image)).unsqueeze(0).permute(0, 3, 1, 2)
    X_embedding = facenet(X_tensor.float())

    return torch.nn.functional.cosine_similarity(target_embedding, X_embedding, dim=1)


In [19]:
comps['lat_comp'].shape

(32, 1, 512)

In [21]:
len(latent_dirs)

32

In [25]:
comps['lat_stdev'].shape

(32,)

In [51]:
#@title Load a component at random
path_to_components="/home/tcalo3/projects/project6/src/modules/components/ganspace/cache/components/stylegan-ffhq_g_mapping.dense7_pca-full_c32_n300000_w.npz"
comps = np.load(path_to_components)

latent_dirs = comps['lat_comp'][:,0,:]
latent_stdevs = comps['lat_stdev']
data_mean = comps['lat_mean']

#load one at random
dims = latent_stdevs.shape[0]
num = np.random.randint(dims)
random_dir = latent_dirs[num]
random_dir_stdev = latent_stdevs[num]
print(f'Loaded Component No. {num}')


Loaded Component No. 28


In [4]:

def reconstruct_and_evaluate(x, target, model, latent_dirs,eval_model, data_mean):
    target_out = model.sample_np(target)
    target_image = Image.fromarray((target_out * 255).astype(np.uint8))
    target_resized_image = target_image.resize((160, 160), Image.NEAREST)
    target_tensor = torch.from_numpy(np.array(target_resized_image)).unsqueeze(0).permute(0, 3, 1, 2)
    target_embedding = eval_model(target_tensor.float())

    x_reconstructed = reconstruct_from_pca(x,latent_dirs,data_mean)
    x_gan = model.sample_np(x_reconstructed.astype(np.float32))
    x_image = Image.fromarray((x_gan * 255).astype(np.uint8))
    x_resized_image = x_image.resize((160, 160), Image.NEAREST)
    x_tensor = torch.from_numpy(np.array(x_resized_image)).unsqueeze(0).permute(0, 3, 1, 2)
    x_embedding = eval_model(x_tensor.float())

    return torch.nn.functional.cosine_similarity(target_embedding, x_embedding, dim=1)

In [41]:
target = model.sample_latent(1).cpu().detach().numpy() 
# Generate initial training data
train_x = sample_pca_space(1,dims,latent_stdevs)

# Parameters
num_arms = dims
max_steps = 1000
c = 1.5

# Initialize variables
Q_a = np.zeros(num_arms)
N_a = np.zeros(num_arms)
total_rewards = 0

q_a = np.array([np.random.normal(0, 1) for _ in range(num_arms)])

# Function to select action using the Upper Confidence Bound algorithm
def select_action(t):
    with np.errstate(divide='ignore', invalid='ignore'):
        upper_confidence_bounds = Q_a + c * np.sqrt(np.log(t) / N_a)
    return np.argmax(upper_confidence_bounds)


In [69]:
target = model.sample_latent(1).cpu().detach().numpy() 
# y_pca = project_to_pca(target,latent_dirs,data_mean)
# y_reconstructed = reconstruct_from_pca(y_pca,latent_dirs,data_mean)
y_gan =  model.sample_np(target.astype(np.float32))
y_image = Image.fromarray((y_gan * 255).astype(np.uint8))


In [58]:
x_pca = sample_pca_space(1,dims,latent_stdevs)
x_reconstructed = reconstruct_from_pca(x_pca,latent_dirs,data_mean)
x_gan = model.sample_np(x_reconstructed.astype(np.float32))
x_image = Image.fromarray((x_gan * 255).astype(np.uint8))

In [68]:
step = 0 
action = select_action(step)
train_x = sample_pca_space(1,dims,latent_stdevs)
train_y = torch.tensor([0])  # Assuming the initial image is favored or assumed as a baseline comparison


UnboundLocalError: local variable 'inputs' referenced before assignment

In [40]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood

def pull_arm(dimension, pca_components, pca_stdevs, data_mean, objective_function, train_x, train_y, bounds):
    # Assume the most recent or best values for other dimensions are stored in train_x
    if train_x is None:
        # Initial sampling for all dimensions
        current_values = torch.zeros(1, pca_components.shape[1])
    else:
        # Start with the last known best values
        current_values = train_x[-1].unsqueeze(0).clone()  

    # Create a GP model from the existing data
    gp_model = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
    fit_gpytorch_model(mll)

    # Define the acquisition function
    EI = ExpectedImprovement(model=gp_model, best_f=train_y.max())

    # Optimize the acquisition function, modifying only the selected dimension
    # Set bounds for only the selected dimension to vary
    lower_bounds, upper_bounds = bounds[:, 0], bounds[:, 1]
    fixed_bounds = torch.stack([lower_bounds, upper_bounds], dim=-1)
    fixed_bounds[:, dimension] = torch.tensor([bounds[dimension, 0], bounds[dimension, 1]])  # Only free this dimension

    # Optimize using the acquisition function
    candidate, _ = optimize_acqf(
        acq_function=EI,
        bounds=fixed_bounds,
        q=1,
        num_restarts=10,
        raw_samples=512,
    )

    # Update only the selected dimension with the new candidate
    current_values[0, dimension] = candidate[0, dimension]

    # Evaluate the objective function using the updated values
    train_x_reconstructed = reconstruct_from_pca(current_values.numpy(), pca_components[:, dimension].T, data_mean)
    reward = objective_function(train_x_reconstructed, data_mean)

    # Update training data
    train_x = torch.cat([train_x, current_values], dim=0)
    train_y = torch.cat([train_y, torch.tensor([reward])], dim=0)

    return reward, current_values



torch.Size([1, 32])

In [17]:

# Parameters
num_arms = dims
max_steps = 1000
c = 1.5

# Initialize variables
Q_a = np.zeros(num_arms)
N_a = np.zeros(num_arms)
total_rewards = 0

q_a = np.array([np.random.normal(0, 1) for _ in range(num_arms)])

def pull_arm(dimension, pca_components, pca_stdevs, data_mean, objective_function, train_x, train_y, bounds, target):
    # Sample initial point in PCA space and modify it along the selected arm
    if train_x is None:
        train_x_dim = torch.zeros(1, pca_components.shape[0])
    else:
        train_x_dim = train_x[-1].clone()

    # Define the pairwise GP model using existing data
    gp_model = PairwiseGP(train_x, train_y)
    
    # Define the acquisition function
    mll = PairwiseLaplaceMarginalLogLikelihood(gp_model.likelihood,gp_model)
    mll = fit_gpytorch_mll(mll)

    # Optimize the acquisition function
    pca_sample = sample_pca_space(1, pca_components.shape[1], pca_stdevs)  # Start with a new sample
    train_x_dim[0, dimension] = pca_sample[0, dimension]  # Adjust only the selected dimension
    candidate, acq_value = optimize_acqf(
        acq_function=mll,
        bounds=torch.tensor([bounds[:, dimension]]).to(torch.float32),
        q=1,
        num_restarts=10,
        raw_samples=512,
    )

    # Use the optimized candidate to update the dimension in train_x
    train_x_dim[0, dimension] = candidate.squeeze()

    # Reconstruct the full sample from PCA and evaluate the objective
    train_x_reconstructed = reconstruct_from_pca(train_x_dim.numpy(), pca_components[:, dimension].T, data_mean)
    reward = objective_function(train_x_reconstructed, target)

    # Update the training data
    if train_x is None:
        train_x = train_x_dim.unsqueeze(0)
        train_y = torch.tensor([reward])
    else:
        train_x = torch.cat([train_x, train_x_dim.unsqueeze(0)], dim=0)
        train_y = torch.cat([train_y, torch.tensor([reward])], dim=0)

    return reward, train_x_dim


# Function to select action using the Upper Confidence Bound algorithm
def select_action(t):
    with np.errstate(divide='ignore', invalid='ignore'):
        upper_confidence_bounds = Q_a + c * np.sqrt(np.log(t) / N_a)
    return np.argmax(upper_confidence_bounds)




In [15]:
target = model.sample_latent(1).cpu().detach().numpy() 
# Generate initial training data
pca_sample = sample_pca_space(1,dims,latent_stdevs)


In [14]:
pca_sample.shape

(1, 32)

In [ ]:

# Running the bandit algorithm
for step in range(1, max_steps):
    action = select_action(step)
    current_reward = pull_arm(action)
    total_rewards[action] += current_reward
    N_a[action] += 1
    Q_a[action] += (current_reward - Q_a[action]) / N_a[action]